In [ ]:
import yaml
from pytorch_lightning.loggers import NeptuneLogger
from Modules.train import DataModule, Model, get_trainer


# Training

In [ ]:
# %%script false --no-raise-error

task = "segmentation"

neptune_logger = NeptuneLogger(
        project="kaori/AISeed",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyZjZiMDA2YS02MDM3LTQxZjQtOTE4YS1jODZkMTJjNGJlMDYifQ==",
        log_model_checkpoints=False,
        tags=[task]
    )

with open("Modules/config.yaml", 'r') as stream:
    PARAMS = yaml.safe_load(stream)
    PARAMS = PARAMS['segment']
    print(PARAMS)



In [ ]:
# %%script false --no-raise-error

import albumentations as A
from albumentations.pytorch import ToTensorV2

img_size = PARAMS['dataset_settings']['img_size']
transforms_img = A.Compose([
    A.RandomResizedCrop(img_size, img_size),  # Random crop and resize
    A.HorizontalFlip(),  # Random horizontal flip
    A.VerticalFlip(),  # Random vertical flip
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),  # Convert to tensor
])

data = DataModule(PARAMS['dataset_settings'], PARAMS['training_settings'], [None, None])

In [ ]:
# %%script false --no-raise-error
neptune_logger.log_hyperparams(params=PARAMS)
model = Model(PARAMS=PARAMS, task = task)
trainer = get_trainer(PARAMS['training_settings'], neptune_logger)
# 
trainer.fit(model, data)

# Testing

In [ ]:
%%script false --no-raise-error
import torch
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import os
from torchvision import transforms

with open("models/class_name.txt", "r", encoding='utf-8') as f:
    class_names = f.read().splitlines()

model_list = [name.split('.')[0] for name in os.listdir("models") if name.endswith('ckpt')]
model_list += ["fcn", "resnet", "fasterrcnn", "maskrcnn"]
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)

def predict(image, model_choice):
    labels, detection = None, None
    with open("Modules/config.yaml", 'r') as stream:
            PARAMS = yaml.safe_load(stream)
            PARAMS = PARAMS['segment']
    PARAMS['architect_settings']['backbone']['is_full'] = True
    if(model_choice == "fcn"):
        PARAMS['architect_settings']['backbone']['name'] = 'fcn-m'    
        model = Model(PARAMS, task="segmentation")
    elif(model_choice == "resnet"):
        PARAMS['architect_settings']['backbone']['name'] = 'resnet-s'    
        model = Model(PARAMS, task="classification")
    elif(model_choice == "fasterrcnn"):
        PARAMS['architect_settings']['backbone']['name'] = 'fasterrcnn-s'    
        model = Model(PARAMS, task="detection")
    elif(model_choice == "maskrcnn"):
        PARAMS['architect_settings']['backbone']['name'] = 'maskrcnn-s'    
        model = Model(PARAMS, task="detection")
    else:
        model = Model.load_from_checkpoint(f"models/{model_choice}.ckpt").cpu()
    model.eval()
    transforms = model.model.preprocess
    tensor_image = transforms(image)
    with torch.no_grad():
        y_hat = model(tensor_image.unsqueeze(0))
        if(model.task == "classification"):
            preds = torch.softmax(y_hat, dim=-1).tolist()
            labels = {class_names[k]: float(v) for k, v in enumerate(preds[0][:-1])}
        elif(model.task == "segmentation"):
            num_classes = y_hat.shape[1]
            masks = y_hat[0]
            classes_masks = masks.argmax(0) == torch.arange(num_classes)[:, None, None]
            tensor_image = inv_normalize(tensor_image)
            detection = draw_segmentation_masks((tensor_image * 255.).to(torch.uint8), 
                                              masks=classes_masks, alpha=.6)
            detection = detection.numpy().transpose(1, 2, 0) / 255.
        elif(model.task == "detection"):
            if("maskrcnn" in model_choice):
                boolean_masks = [out['masks'][out['scores'] > .75] > 0.5
                                for out in y_hat][0]
                detection = draw_segmentation_masks((tensor_image * 255.).to(torch.uint8),
                                                    boolean_masks.squeeze(1), alpha=0.8)
                detection = detection.numpy().transpose(1, 2, 0) / 255.
            else:
                detection = draw_bounding_boxes((tensor_image * 255.).to(torch.uint8), 
                                                    boxes=y_hat[0]["boxes"][:5],
                                                    colors="red",
                                                    width=5)
                detection = detection.numpy().transpose(1, 2, 0) / 255.

    return labels, detection

In [ ]:
%%script false --no-raise-error

import gradio as gr

title = "Application Demo "
description = "# A Demo of Wrapping Pretrained Networks"
example_list = [["examples/" + example] for example in os.listdir("examples")]


with gr.Blocks() as demo:
    demo.title = title
    gr.Markdown(description)
    with gr.Row():
        with gr.Column():
            model = gr.Dropdown(model_list, label="Select Model", interactive=True)
            im = gr.Image(type="pil", label="input image")
            label_conv = gr.Label(label="Predictions", num_top_classes=4)
        with gr.Column():
            im_detection = gr.Image(type="pil", label="Detection")
            btn = gr.Button(value="predict")
    btn.click(predict, inputs=[im, model], outputs=[label_conv, im_detection])
    gr.Examples(examples=example_list, inputs=[im, model], outputs=[label_conv, im_detection])
      

demo.launch(share=True)